# <center> transformer框架 </center>
by Hyr1sky_He

Link to the lecture [68 Transformer【动手学深度学习v2】](https://www.bilibili.com/video/BV1Kq4y1H7FL/?share_source=copy_web&vd_source=7d2cf6f427cab8ff5afa3cb534b98123)

## Multihead Attention

In [2]:
import math
import torch
from torch import nn
from d2l import torch as d2l

if torch.cuda.is_available():
    device = torch.device('cuda')

In [10]:
def transpose_qkv(X, num_heads):
    """Note
    For parrallel computation, we can concat the heads together
    INPUT: X.shape = (batch_size, num_steps, num_hiddens)
    OUTPUT: X.shape = (batch_size * num_heads, num_steps, num_hiddens/num_heads)
    combine the num_heads and num_hiddens/num_heads together
    """
    X = X.reshape(X.shape[0], X.shape[1], num_heads, -1)
    X = X.permute(0, 2, 1, 3)
    return X.reshape(-1, X.shape[2], X.shape[3])

def transpose_output(X, num_heads):
    """Note
    Inverse of transpose_qkv
    INPUT: X.shape = (batch_size * num_heads, num_steps, num_hiddens/num_heads)
    OUTPUT: X.shape = (batch_size, num_steps, num_hiddens)
    """
    X = X.reshape(-1, num_heads, X.shape[1], X.shape[2])
    X = X.permute(0, 2, 1, 3)
    return X.reshape(X.shape[0], X.shape[1], -1)

In [9]:
class multiheadAttention (nn.Module):
    def __init__(self, query_size, key_size, value_size,
                 num_hiddens, num_heads, dropout, bias=False, 
                 *args, **kwargs) -> None:
        super(multiheadAttention, self).__init__(*args, **kwargs)
        self.num_heads = num_heads
        self.attention = d2l.DotProductAttention(dropout)
        self.W_q = nn.Linear(query_size, num_hiddens, bias=bias)
        self.W_k = nn.Linear(key_size, num_hiddens, bias=bias)
        self.W_v = nn.Linear(value_size, num_hiddens, bias=bias)
        self.W_o = nn.Linear(num_hiddens, num_hiddens, bias=bias)

    def forward(self, queries, keys, values, valid_lens):
        """Note
        concat all the heads together for matrix multiplication
        """
        queries = transpose_qkv(self.W_q(queries), self.num_heads)
        keys = transpose_qkv(self.W_k(keys), self.num_heads)
        values = transpose_qkv(self.W_v(values), self.num_heads)
        if valid_lens is not None:
            valid_lens = torch.repeat_interleave(valid_lens, repeats=self.num_heads, dim=0)
        output = self.attention(queries, keys, values, valid_lens)
        output_concat = transpose_output(output, self.num_heads)
        return self.W_o(output_concat)

In [11]:
# Test
num_hiddens, num_heads = 100, 5 
attention = multiheadAttention(num_hiddens, num_hiddens, num_hiddens, num_hiddens, num_heads, 0.5)
attention.eval()
print(attention)

batch_size, num_steps = 2, 4
num_kvpairs, valid_lens = 2, torch.tensor([3, 2])
X = torch.ones((batch_size, num_steps, num_hiddens))
Y = torch.ones((batch_size, num_steps, num_hiddens))
attention(X, Y, Y, valid_lens).shape

multiheadAttention(
  (attention): DotProductAttention(
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (W_q): Linear(in_features=100, out_features=100, bias=False)
  (W_k): Linear(in_features=100, out_features=100, bias=False)
  (W_v): Linear(in_features=100, out_features=100, bias=False)
  (W_o): Linear(in_features=100, out_features=100, bias=False)
)


torch.Size([2, 4, 100])

## Transformer